In [3]:
import requests
from requests import Response
from pydantic import RootModel,BaseModel,Field,field_validator
from datetime import datetime
import pandas as pd 

class Site (BaseModel):
    行政區域:str = Field(alias='sarea')
    總數量:int = Field(alias='total')
    可借數量:int = Field(alias='available_rent_bikes') 
    可還數量:int = Field(alias='available_return_bikes')
    時間:datetime = Field(alias='mday')

    @field_validator('可借數量','可還數量',mode='before')
    @classmethod
    def whitespace_to_zero(cls, value: str) -> str:
        return '0.0' if value == '' else value
    
class Youbike(RootModel):
    root:list[Site]

try:
    youbike_url:Response= requests.get('https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json')
    youbike_url.raise_for_status()
except Exception as e:
    print(e)
else:
    data = Youbike.model_validate_json(youbike_url.text)
    all_sites = data.model_dump()

df = pd.DataFrame(all_sites)
df

,行政區域,總數量,可借數量,可還數量,時間
0,大安區,28,0,28,2024-08-06 20:21:14
1,大安區,21,0,21,2024-08-06 20:19:14
2,大安區,16,2,14,2024-08-06 20:13:20
3,大安區,11,4,7,2024-08-06 20:21:20
4,大安區,16,1,15,2024-08-06 20:23:10
...,...,...,...,...,...
1433,臺大公館校區,30,1,29,2024-08-06 20:21:20
1434,臺大公館校區,35,2,33,2024-08-06 20:23:15
1435,臺大公館校區,24,0,24,2024-08-06 19:33:47
1436,臺大公館校區,40,7,33,2024-08-06 20:22:21


In [4]:
df.groupby(by='行政區域')[['總數量','可借數量','可還數量']].sum()

,總數量,可借數量,可還數量
行政區域,,,
中山區,3904,1240,2593
中正區,3267,840,2378
信義區,3360,839,2431
內湖區,3521,1096,2394
北投區,2537,947,1573
南港區,2317,798,1500
士林區,2962,1164,1688
大同區,1629,633,984
大安區,4986,1529,3334


In [5]:
df1 = df.groupby(by='行政區域')[['總數量','可借數量','可還數量']].agg([('車輛數','sum'),('站點數','count')])
df1

總數量       可借數量       可還數量     
         車輛數  站點數   車輛數  站點數   車輛數  站點數
行政區域                                   
中山區     3904  159  1240  159  2593  159
中正區     3267  122   840  122  2378  122
信義區     3360  107   839  107  2431  107
內湖區     3521  169  1096  169  2394  169
北投區     2537   96   947   96  1573   96
南港區     2317   92   798   92  1500   92
士林區     2962  128  1164  128  1688  128
大同區     1629   62   633   62   984   62
大安區     4986  185  1529  185  3334  185
文山區     2216   97   869   97  1330   97
松山區     2366   88   890   88  1458   88
臺大公館校區  1642   53   271   53  1369   53
萬華區     2057   80   850   80  1133   80